<a href="https://colab.research.google.com/github/connixu/studious_pancake/blob/main/Fortune_1000_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Web Scraping Excercise - Fortune 1000 

This portion of the code is to be run to set up some of the basic requirements in order to get data from the Fortune website. I have to download packages to work the remaining code.

In [1]:
!pip install bs4
import os
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import lxml
import numbers
from bs4 import BeautifulSoup
import requests
import json

req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 2022

### Scrape JSON from website 

This code is derived from a [Kaggle coder](https://www.kaggle.com/datasets/winston56/fortune-500-data-2021?resource=download) but should continue to work as we update the URL of the list site as long as the site format and sourcing does not change substantially 




In [2]:
url_2="https://fortune.com/wp-json/irving/v1/data/franchise-search-results?list_id=3287962&token=Zm9ydHVuZTpCcHNyZmtNZCN5SndjWkkhNHFqMndEOTM="

In [35]:
# request the URL and parse the JSON
response = requests.get(url_2)
response.raise_for_status() # raise exception if invalid response
comps = response.json()
print(len(comps))
comp = comps[1]
print(len(comp))

2
2


In [36]:
# prettify json
format_json = json.dumps(comp, sort_keys=True, indent=5)
print(format_json[:1000])

{
     "items": [
          {
               "fields": [
                    {
                         "key": "rank",
                         "value": "941"
                    },
                    {
                         "key": "f500_revenues",
                         "value": "2399.8"
                    },
                    {
                         "key": "revchange",
                         "value": "29.5"
                    },
                    {
                         "key": "f500_profits",
                         "value": "203.3"
                    },
                    {
                         "key": "prftchange",
                         "value": "168.6"
                    },
                    {
                         "key": "assets",
                         "value": "5138.4"
                    },
                    {
                         "key": "f500_mktval",
                         "value": "5434.8"
                    },
                 

### Code below is to get data format - do not bother to run these if following subsection is not causing issues

In [52]:
# select the third company (for illustrative example)
n_comp = comp['items'][1]
print(n_comp)

{'fields': [{'key': 'rank', 'value': '951'}, {'key': 'f500_revenues', 'value': '2335'}, {'key': 'revchange', 'value': '161.2'}, {'key': 'f500_profits', 'value': '176'}, {'key': 'prftchange', 'value': ''}, {'key': 'assets', 'value': '8008'}, {'key': 'f500_mktval', 'value': '6247.9'}, {'key': 'rankchange1000', 'value': ''}, {'key': 'f500_employees', 'value': '13000'}, {'key': 'rankchange', 'value': ''}, {'key': 'name', 'value': 'Hilton Grand Vacations'}, {'key': 'sector', 'value': 'Hotels, Restaurants & Leisure'}, {'key': 'f500_industry', 'value': 'Hotels, Casinos, Resorts'}, {'key': 'hqcity', 'value': 'Orlando'}, {'key': 'hqstate', 'value': 'FL'}, {'key': 'rankgain', 'value': ''}, {'key': 'rankdrop', 'value': ''}, {'key': 'newcomer', 'value': 'no'}, {'key': 'profitable', 'value': 'yes'}, {'key': 'ceofounder', 'value': 'no'}, {'key': 'ceowoman', 'value': 'no'}, {'key': 'jobgrowth', 'value': 'yes'}, {'key': 'global500-y-n', 'value': 'no'}, {'key': 'best-companies-y-n', 'value': 'no'}, {'f

In [53]:
n_comp.keys()

dict_keys(['fields', 'permalink'])

In [54]:
new_comp = n_comp['permalink']
print(new_comp)

https://fortune.com/company/hilton-grand-vacations/fortune500/


In [47]:
my_comp = n_comp['fields']
print(my_comp)


[{'key': 'rank', 'value': '954'}, {'key': 'f500_revenues', 'value': '2313.4'}, {'key': 'revchange', 'value': '66.9'}, {'key': 'f500_profits', 'value': '725.7'}, {'key': 'prftchange', 'value': '132'}, {'key': 'assets', 'value': '1545.1'}, {'key': 'f500_mktval', 'value': '4676.2'}, {'key': 'rankchange1000', 'value': ''}, {'key': 'f500_employees', 'value': '5770'}, {'key': 'rankchange', 'value': ''}, {'key': 'name', 'value': 'Crocs'}, {'key': 'sector', 'value': 'Apparel'}, {'key': 'f500_industry', 'value': 'Apparel'}, {'key': 'hqcity', 'value': 'Broomfield'}, {'key': 'hqstate', 'value': 'CO'}, {'key': 'rankgain', 'value': ''}, {'key': 'rankdrop', 'value': ''}, {'key': 'newcomer', 'value': 'no'}, {'key': 'profitable', 'value': 'yes'}, {'key': 'ceofounder', 'value': 'no'}, {'key': 'ceowoman', 'value': 'no'}, {'key': 'jobgrowth', 'value': 'yes'}, {'key': 'global500-y-n', 'value': 'no'}, {'key': 'best-companies-y-n', 'value': 'no'}, {'fieldMeta': {'title': 'Company', 'type': 'Text', 'importFi

In [55]:
fields = []

for j in comp['items']:
    
    #for key, value in comp:
    field = j['fields']
    fields.append(field)
        
print(len(fields))

1000


In [13]:
df = pd.DataFrame()

In [11]:
def gather_data(name, col):

    lst = []
    val = ""
    for i in fields:
        for j in i:
            if j['key'] == name:
                val = j['value']
            else:
                pass
        
        lst.append(val)
        
    df[col] = lst
    
    return df

In [14]:
keys = ['name', 'rank', 'rankchange', 'f500_revenues', 'f500_profits', 
        'f500_ employees', 'sector', 'hqcity', 'hqstate', 'newcomer', 
        'ceofounder', 'ceowoman', 'profitable']

cols = ['company', 'rank', 'rank_change', 'revenue', 'profit',
        'num. of employees', 'sector', 'city', 'state', 'newcomer', 
        'ceo_founder', 'ceo_woman', 'profitable']


for x,y in zip(keys, cols):
    f500_df = gather_data(x, y)

In [15]:
print(f500_df.shape)
f500_df.head()

(1000, 13)


,company,rank,rank_change,revenue,profit,num. of employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable
0,Element Solutions,941,,2399.8,203.3,,Chemicals,Fort Lauderdale,FL,no,no,no,yes
1,Hilton Grand Vacations,951,,2335,176,,"Hotels, Restaurants & Leisure",Orlando,FL,no,no,no,yes
2,Crocs,954,,2313.4,725.7,,Apparel,Broomfield,CO,no,no,no,yes
3,Olympic Steel,955,,2312.3,121.1,,Materials,Highland Hills,OH,no,no,no,yes
4,Signature Bank,956,,2311.3,918.4,,Financials,New York,NY,no,yes,no,yes


In [17]:
f500_df['rank'] = f500_df['rank'].astype('int')

f500_df.sort_values(by = 'rank')
f500_df[f500_df['rank_change']!='']

,company,rank,rank_change,revenue,profit,num. of employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable
503,Frontier Communications,499,-97,6411,,,Telecommunications,Norwalk,CT,no,no,no,no
504,Ameren,500,-31,6394,990,,Energy,St. Louis,MO,no,no,no,yes
507,Hasbro,496,-2,6420.4,428.7,,Household Products,Pawtucket,RI,no,no,no,yes
508,Roper Technologies,497,-9,6415.8,1152.6,,Technology,Sarasota,FL,no,no,no,yes
510,Yum Brands,490,-12,6584,1575,,"Hotels, Restaurants & Leisure",Louisville,KY,no,no,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,Alphabet,8,1,257637,76033,,Technology,Mountain View,CA,no,no,no,yes
991,McKesson,9,-2,238228,-4539,,Health Care,Irving,TX,no,no,no,no
992,AmerisourceBergen,10,-2,213988.8,1539.9,,Health Care,Conshohocken,PA,no,no,no,yes
994,Exxon Mobil,6,4,285640,23040,,Energy,Irving,TX,no,no,no,yes


In [23]:
f500_df.sort_values(by = 'rank').head(10)


,company,rank,rank_change,revenue,profit,num. of employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable
999,Walmart,1,,572754,13673,,Retailing,Bentonville,AR,no,no,no,yes
996,Amazon,2,,469822,33364,,Retailing,Seattle,WA,no,no,no,yes
997,Apple,3,,365817,94680,,Technology,Cupertino,CA,no,no,no,yes
998,CVS Health,4,,292111,7910,,Health Care,Woonsocket,RI,no,no,yes,yes
993,UnitedHealth Group,5,,287597,17285,,Health Care,Minnetonka,MN,no,no,no,yes
994,Exxon Mobil,6,4,285640,23040,,Energy,Irving,TX,no,no,no,yes
995,Berkshire Hathaway,7,-1,276094,89795,,Financials,Omaha,NE,no,no,no,yes
990,Alphabet,8,1,257637,76033,,Technology,Mountain View,CA,no,no,no,yes
991,McKesson,9,-2,238228,-4539,,Health Care,Irving,TX,no,no,no,no
992,AmerisourceBergen,10,-2,213988.8,1539.9,,Health Care,Conshohocken,PA,no,no,no,yes


## 2021


Same code as earlier but for 2021 - this one was the same as on Kaggle.

In [ ]:
page_url = 'https://content.fortune.com/wp-json/irving/v1/data/franchise-search-results?list_id=2814606&token=Zm9ydHVuZTpCcHNyZmtNZCN5SndjWkkhNHFqMndEOTM='

In [ ]:
response = requests.get(page_url)
response.raise_for_status() # raise exception if invalid response
comps = response.json()
comp = comps[1]


2


In [ ]:
fields = []

for j in comp['items']:
    
    #for key, value in comp:
    field = j['fields']
    fields.append(field)
        
print(len(fields))

1000


In [ ]:
df = pd.DataFrame()

In [ ]:
def gather_data(name, col):

    lst = []
    val = ""
    for i in fields:
        for j in i:
            if j['key'] == name:
                val = j['value']
            else:
                pass
        
        lst.append(val)
        
    df[col] = lst
    
    return df

In [ ]:
keys = ['name', 'rank', 'rankchange', 'f500_revenues', 'f500_profits', 
        'f500_ employees', 'sector', 'hqcity', 'hqstate', 'newcomer', 
        'ceofounder', 'ceowoman', 'profitable']

cols = ['company', 'rank', 'rank_change', 'revenue', 'profit',
        'num. of employees', 'sector', 'city', 'state', 'newcomer', 
        'ceo_founder', 'ceo_woman', 'profitable']


for x,y in zip(keys, cols):
    f500_df_21 = gather_data(x, y)

In [ ]:
f500_df_21['rank'] = f500_df_21['rank'].astype('int')

f500_df_21.sort_values(by = 'rank')
f500_df_21[f500_df_21['rank_change']!='']

,company,rank,rank_change,revenue,profit,num. of employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable
205,Wynn Resorts,454,-6,6611.1,123,30200,"Hotels, Restaurants & Leisure",Las Vegas,NV,no,no,no,yes
222,Spirit AeroSystems Holdings,406,14,7863.1,530.1,18200,Aerospace & Defense,Wichita,KS,no,no,no,yes
244,Voya Financial,353,-10,8942,-351,6000,Financials,New York,NY,no,no,no,no
245,Bread Financial Holdings,456,-66,6580.9,278,8500,Business Services,Columbus,OH,no,no,no,yes
257,Ralph Lauren,469,4,6313,430.9,18650,Apparel,New York,NY,no,no,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Anixter International,359,5,8845.6,262.9,9400,Wholesalers,Glenview,IL,no,no,no,yes
996,Caesars Holdings,363,2,8742,-1195,64000,"Hotels, Restaurants & Leisure",Las Vegas,NV,no,no,no,no
997,Calpine,319,11,10072,770,2256,Energy,Houston,TX,no,no,no,yes
998,J.C. Penney,286,-25,11167,-268,90000,Retailing,Plano,TX,no,no,no,no
